# NC Water Scraper
This notebook consolidates water withdrawal, discharge, and transfer data from NCDEQs Water Withdrawal & Transfer Registry ([link](http://www.ncwater.org/Permits_and_Registration/Water_Withdrawal_and_Transfer_Registration/report)). The sequence of analysis is as follows:
* First data from the registry's front page (link above) is scraped into a dataframe listing Registered Owner, Facility Name, Status, and Facility ID. 
* Then, using each entry's facility ID to access its annual report, data are scraped - one facility and one year at a time - to compile a table listing monthly withdrawals, discharges, and transfers for a set of years. 

In [1]:
#import libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

## Step 1. Create a dataframe of sites in the registry

In [2]:
#Extract the contents of the base web page into a 'soup' object for scraping
baseURL = 'http://www.ncwater.org/Permits_and_Registration/Water_Withdrawal_and_Transfer_Registration/report'
r = requests.get(baseURL)
soup = BeautifulSoup(r.text,'lxml')

The following works with the current format of the web page. If it changes, this may have to be revised. Here we select the items in the web form containing the data we want to extract. 

In [3]:
#Get the main table, identified with the id=main
trTable = soup.find_all(id="main")[0]

In [4]:
#Get the second table contained in the table selected above
dataTable = trTable.find_all('table')[1]

In [5]:
#Get all rows in the table selected above; these contain the data we want
rows = dataTable.find_all('tr')

In [6]:
#Initialize the dataframe that will hold our data
colNames = ['Owner','Name','Status','Code']
dfSites = pd.DataFrame(columns=colNames)
dfSites.head()

,Owner,Name,Status,Code


In [7]:
#Loop through each row (skipping the first, which contains headers), extracting data into our data frame
for row in rows[1:]:
    
    #Create a collection of columns for the current row
    columns = row.find_all('td')
    
    #Construct a dictionary of the items we want
    dictR = {'Owner':columns[0].string,
             'Name':columns[1].string,
             'Status':columns[2].string,
             'Code':columns[3].find("a")['href'].split("/")[-2]}
    
    #Append these data to our dataframe
    dfSites = dfSites.append(dictR,ignore_index=True)
    
dfSites.head()

,Owner,Name,Status,Code
0,3M Company,3M Pittsboro Mine,Completed,0831-0001
1,AAAAcme,AAcme,Draft,0847-0001
2,"Alamac American Knits, LLC",Alamac American Knits,Completed,0292-0001
3,Alamance Country Club,Alamance Country Club,Completed,0043-0001
4,"American & Efird, Inc.",Plant 15,Completed,0004-0001


In [8]:
#Save contents to a file...
dfSites.to_csv("../../Data/NCDEQ/WithdrawalMaster.csv",index=False)

## Step 2. Extract report data for each site

In [9]:
def unstackTable(dfStacked):
    '''
    Unstacks monthly tables presented in 2-column formats into a 
    single column format. For example:
    | Jan | Jul | 
    | Feb | Aug |
    | Mar | Sep | 
    | Apr | Oct | 
    | May | Nov | 
    | Jun | Dec | 
    is converted to a single column with associated data attached. 
    '''
    #Copy the table
    df2 = dfStacked.copy(deep=True)
    
    #Convert the first row to columns, then drop the row
    colNames = df2.iloc[0]
    df2.columns = colNames
    df2.drop(0,inplace=True)

    #Convert two column format to one
    df2a = df2.iloc[:,:4]
    df2b = df2.iloc[:,4:]
    df2 = df2a.append(df2b)

    #Set month to be the index
    df2.set_index("Month",inplace=True)

    #Convert data types for columns 2, 3, and 4 (days, avg, max)
    df2.iloc[:,0] = df2.iloc[:,0].fillna(0).astype(int)
    df2.iloc[:,1] = df2.iloc[:,1].astype(float)
    df2.iloc[:,2] = df2.iloc[:,2].astype(float)
    
    #Return the table
    return df2

In [10]:
def ScrapeSite(siteID, year, first=False):
    
    #--DATA EXTRACTION--
    #Construct the URL
    siteURL = 'http://www.ncwater.org/Permits_and_Registration/Water_Withdrawal_and_Transfer_Registration/report/view/{0}/{1}'.format(siteID,year)

    #Extract all tables from the URL into a collection of dataframes
    dfs = pd.read_html(siteURL,na_values='NaN')

    #Separate tables into labeled variables, unstacking as needed
    dfFacility = dfs[2]                  # Information on the facility
    dfWithdrawal = unstackTable(dfs[3])  # Monthly withdrawal data   
    dfSource = dfs[4]                    # Information on where water was drawn
    dfDischarge = unstackTable(dfs[5])   # Monthly discharge data 
    dfDischargeMethod = dfs[6]           # Information on type and amounts of discharge
    dfTransferDescription = dfs[7]       # Information on source and destination of transfers
    dfTransfer= unstackTable(dfs[8])     # Monthly transfer data

    #Extract facility information into variables
    registrant = dfFacility.iloc[0,1]
    facility_name = dfFacility.iloc[0,1]
    county = dfFacility.iloc[2,1]
    subbasin= dfFacility.iloc[2,3]
    facility_type = dfFacility.iloc[1,3]
    
    #--MONTHLY VOLUME DATA----------------------
    #Combine monthly withdrawal, discharge, and transfer tables
    dfSiteData = pd.concat([dfWithdrawal,dfDischarge,dfTransfer], axis=1).reset_index()

    #Add site information as columns
    dfSiteData['SiteID'] = siteID
    dfSiteData['Year'] = year
    dfSiteData['Registrant'] = registrant
    dfSiteData['Facility'] = facility_name
    dfSiteData['Type'] = facility_type
    dfSiteData['County'] = county
    dfSiteData['Subbasin'] = subbasin

    #Rearrange columns
    columns = dfSiteData.columns.tolist()[10:] + dfSiteData.columns.tolist()[:10]
    dfSiteData = dfSiteData[columns]

    #--WITHDRAWAL INFO--------------------------
    dfSource = dfs[4].copy(deep=True)
    dfSource.columns = ('Name','Type','AvgDaily','DaysUsed','Capacity_MGD')
    dfSource.drop(0,inplace=True)
    dfSource.insert(0,'SiteID',siteID)
    dfSource.insert(1,'Year',year)    
    dfSource.insert(2,'FacilityType',facility_type)
    dfSource.insert(3,'County',county)
    dfSource.insert(4,'Subbasin',subbasin)

    #--DISCHARGE INFO-------------------------
    dfDischargeMethod = dfs[6].copy(deep=True)
    dfDischargeMethod.columns = ('Permit','Type','AvgDaily','DaysUsed','Capacity_MGD')
    dfDischargeMethod.drop(0,inplace=True)
    dfDischargeMethod.insert(0,'SiteID',siteID)
    dfDischargeMethod.insert(1,'Year',year)
    dfDischargeMethod.insert(2,'FacilityType',facility_type)
    dfDischargeMethod.insert(3,'County',county)
    dfDischargeMethod.insert(4,'Subbasin',subbasin)
    
    #--TRANSFER INFO------------------------------
    dfTransferDescription = dfs[7].copy(deep=True)
    dfTransferDescription.columns = ('Description','SourceBasin','ReceivingBasin','Capacity')
    dfTransferDescription.drop(0,inplace=True)
    dfTransferDescription.insert(0,'SiteID',siteID)
    dfTransferDescription.insert(1,'Year',year)
    dfTransferDescription.insert(2,'FacilityType',facility_type)
    dfTransferDescription.insert(3,'County',county)
    dfTransferDescription.insert(4,'Subbasin',subbasin)    

    #-WRITE DATA TO OUTPUT FILES------------------
    outCSV1 = "../../Data/NCDEQ/MonthlyVolumeData.csv"
    outCSV2 = "../../Data/NCDEQ/WithdrawalSourceData.csv"
    outCSV3 = "../../Data/NCDEQ/DischargeMethods.csv"
    outCSV4 = "../../Data/NCDEQ/TransferInfo.csv"
    
    #If this is the first table, write to new csv files
    if first:
        outputType = 'w' #Write to new file
        head = True      #Include header row
    else: 
        outputType = 'a' #Append to existing file
        head = False     #Don't include headers
        
    #Write monthly volume data to new file
    with open(outCSV1,outputType) as outFile:
        dfSiteData.to_csv(outFile,header=head,index=False)

    #Write source info data to new file  
    with open(outCSV2, outputType) as outFile:
        dfSource.to_csv(outFile,header=head,index=False)

    #Write discharge info data to new file  
    with open(outCSV3, outputType) as outFile:
        dfDischargeMethod.to_csv(outFile,header=head,index=False)

    #Write transfer info data to new file  
    with open(outCSV4, outputType) as outFile:
        dfTransferDescription.to_csv(outFile,header=head,index=False)
            
    return (outCSV1,outCSV2, outCSV3, outCSV4)

In [11]:
#Set flag for the first file (to create a new output)
firstFile = True

#Loop through each site ID and scrape it's data
for index, row in dfSites.iterrows():
    
    #Skip draft data
    if row['Status'] == 'Draft': continue
        
    #Get the code and loop through years
    siteID = row['Code']
    print(index,siteID,end=': ')

    #Loop through years 2010 to 2017 and scrape the data
    for year in range(2010,2018):
        print(year,end='...')
        outFiles = ScrapeSite(siteID,year,first=firstFile)
        firstFile = False
    print()       

0 0831-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
2 0292-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
3 0043-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
4 0004-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
5 0823-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
6 0772-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
7 0218-0016: 2010...2011...2012...2013...2014...2015...2016...2017...
8 0218-0017: 2010...2011...2012...2013...2014...2015...2016...2017...
9 0218-0018: 2010...2011...2012...2013...2014...2015...2016...2017...
10 0218-0019: 2010...2011...2012...2013...2014...2015...2016...2017...
11 0218-0020: 2010...2011...2012...2013...2014...2015...2016...2017...
12 0218-0021: 2010...2011...2012...2013...2014...2015...2016...2017...
13 0218-0022: 2010...2011...2012...2013...2014...2015...2016...2017...
14 0218-0013: 2010...2011...2012...2013...2014...2015...2016...2017...
15 0218-0023: 2

125 0218-0130: 2010...2011...2012...2013...2014...2015...2016...2017...
126 0218-0131: 2010...2011...2012...2013...2014...2015...2016...2017...
127 0218-0132: 2010...2011...2012...2013...2014...2015...2016...2017...
128 0218-0134: 2010...2011...2012...2013...2014...2015...2016...2017...
129 0218-0135: 2010...2011...2012...2013...2014...2015...2016...2017...
130 0218-0136: 2010...2011...2012...2013...2014...2015...2016...2017...
131 0218-0138: 2010...2011...2012...2013...2014...2015...2016...2017...
132 0218-0140: 2010...2011...2012...2013...2014...2015...2016...2017...
133 0218-0139: 2010...2011...2012...2013...2014...2015...2016...2017...
135 0218-0143: 2010...2011...2012...2013...2014...2015...2016...2017...
136 0218-0144: 2010...2011...2012...2013...2014...2015...2016...2017...
137 0218-0145: 2010...2011...2012...2013...2014...2015...2016...2017...
138 0218-0146: 2010...2011...2012...2013...2014...2015...2016...2017...
139 0218-0147: 2010...2011...2012...2013...2014...2015...2016...

245 0218-0253: 2010...2011...2012...2013...2014...2015...2016...2017...
246 0218-0254: 2010...2011...2012...2013...2014...2015...2016...2017...
247 0218-0255: 2010...2011...2012...2013...2014...2015...2016...2017...
248 0218-0256: 2010...2011...2012...2013...2014...2015...2016...2017...
249 0218-0257: 2010...2011...2012...2013...2014...2015...2016...2017...
250 0218-0258: 2010...2011...2012...2013...2014...2015...2016...2017...
251 0218-0260: 2010...2011...2012...2013...2014...2015...2016...2017...
252 0218-0261: 2010...2011...2012...2013...2014...2015...2016...2017...
253 0218-0262: 2010...2011...2012...2013...2014...2015...2016...2017...
254 0218-0263: 2010...2011...2012...2013...2014...2015...2016...2017...
255 0218-0265: 2010...2011...2012...2013...2014...2015...2016...2017...
256 0218-0266: 2010...2011...2012...2013...2014...2015...2016...2017...
257 0218-0267: 2010...2011...2012...2013...2014...2015...2016...2017...
258 0218-0268: 2010...2011...2012...2013...2014...2015...2016...

360 0218-0369: 2010...2011...2012...2013...2014...2015...2016...2017...
361 0218-0370: 2010...2011...2012...2013...2014...2015...2016...2017...
362 0218-0371: 2010...2011...2012...2013...2014...2015...2016...2017...
363 0218-0372: 2010...2011...2012...2013...2014...2015...2016...2017...
364 0218-0373: 2010...2011...2012...2013...2014...2015...2016...2017...
365 0218-0374: 2010...2011...2012...2013...2014...2015...2016...2017...
366 0218-0375: 2010...2011...2012...2013...2014...2015...2016...2017...
367 0218-0010: 2010...2011...2012...2013...2014...2015...2016...2017...
368 0218-0376: 2010...2011...2012...2013...2014...2015...2016...2017...
369 0218-0377: 2010...2011...2012...2013...2014...2015...2016...2017...
370 0218-0378: 2010...2011...2012...2013...2014...2015...2016...2017...
371 0218-0379: 2010...2011...2012...2013...2014...2015...2016...2017...
373 0218-0380: 2010...2011...2012...2013...2014...2015...2016...2017...
374 0218-0381: 2010...2011...2012...2013...2014...2015...2016...

481 0218-0491: 2010...2011...2012...2013...2014...2015...2016...2017...
482 0218-0492: 2010...2011...2012...2013...2014...2015...2016...2017...
483 0218-0494: 2010...2011...2012...2013...2014...2015...2016...2017...
484 0218-0495: 2010...2011...2012...2013...2014...2015...2016...2017...
485 0218-0497: 2010...2011...2012...2013...2014...2015...2016...2017...
486 0218-0498: 2010...2011...2012...2013...2014...2015...2016...2017...
487 0218-0499: 2010...2011...2012...2013...2014...2015...2016...2017...
489 0218-0502: 2010...2011...2012...2013...2014...2015...2016...2017...
490 0218-0503: 2010...2011...2012...2013...2014...2015...2016...2017...
491 0218-0505: 2010...2011...2012...2013...2014...2015...2016...2017...
492 0218-0507: 2010...2011...2012...2013...2014...2015...2016...2017...
494 0218-0509: 2010...2011...2012...2013...2014...2015...2016...2017...
495 0218-0510: 2010...2011...2012...2013...2014...2015...2016...2017...
496 0218-0511: 2010...2011...2012...2013...2014...2015...2016...

...

721 0104-0002: 2010...2011...2012...2013...2014...2015...2016...2017...
722 0104-0003: 2010...2011...2012...2013...2014...2015...2016...2017...
723 0104-0004: 2010...2011...2012...2013...2014...2015...2016...2017...
724 0104-0007: 2010...2011...2012...2013...2014...2015...2016...2017...
725 0104-0005: 2010...2011...2012...2013...2014...2015...2016...2017...
726 0104-0006: 2010...2011...2012...2013...2014...2015...2016...2017...
727 0718-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
728 0824-0002: 2010...2011...2012...2013...2014...2015...2016...2017...
729 0824-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
730 0372-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
731 0241-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
732 0695-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
733 0672-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
734 0785-0001: 2010...2011...2012...2013...2014...2015...2016...

834 0411-0002: 2010...2011...2012...2013...2014...2015...2016...2017...
835 0411-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
836 0045-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
837 0771-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
838 0764-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
839 0736-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
840 0427-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
842 0609-0002: 2010...2011...2012...2013...2014...2015...2016...2017...
843 0382-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
844 0779-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
845 0680-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
846 0664-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
847 0194-0004: 2010...2011...2012...2013...2014...2015...2016...2017...
848 0223-0001: 2010...2011...2012...2013...2014...2015...2016...

949 0620-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
950 0066-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
951 0854-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
952 0358-0002: 2010...2011...2012...2013...2014...2015...2016...2017...
953 0041-0003: 2010...2011...2012...2013...2014...2015...2016...2017...
955 0683-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
956 0670-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
957 0011-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
958 0826-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
959 0618-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
960 0037-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
961 0630-0002: 2010...2011...2012...2013...2014...2015...2016...2017...
962 0630-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
963 0351-0001: 2010...2011...2012...2013...2014...2015...2016...

1063 0756-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
1064 0150-0006: 2010...2011...2012...2013...2014...2015...2016...2017...
1065 0150-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
1066 0150-0002: 2010...2011...2012...2013...2014...2015...2016...2017...
1067 0150-0003: 2010...2011...2012...2013...2014...2015...2016...2017...
1068 0150-0008: 2010...2011...2012...2013...2014...2015...2016...2017...
1069 0150-0005: 2010...2011...2012...2013...2014...2015...2016...2017...
1070 0150-0004: 2010...2011...2012...2013...2014...2015...2016...2017...
1071 0150-0007: 2010...2011...2012...2013...2014...2015...2016...2017...
1072 0742-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
1073 0742-0003: 2010...2011...2012...2013...2014...2015...2016...2017...
1074 0742-0002: 2010...2011...2012...2013...2014...2015...2016...2017...
1075 0789-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
1076 0858-0001: 2010...2011...2012...2013...2014...

1175 0199-0030: 2010...2011...2012...2013...2014...2015...2016...2017...
1176 0199-0015: 2010...2011...2012...2013...2014...2015...2016...2017...
1177 0685-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
1178 0684-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
1179 0202-0003: 2010...2011...2012...2013...2014...2015...2016...2017...
1180 0202-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
1181 0202-0002: 2010...2011...2012...2013...2014...2015...2016...2017...
1182 0202-0004: 2010...2011...2012...2013...2014...2015...2016...2017...
1183 0020-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
1184 0640-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
1185 0054-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
1186 0839-0002: 2010...2011...2012...2013...2014...2015...2016...2017...
1187 0839-0001: 2010...2011...2012...2013...2014...2015...2016...2017...
1188 0207-0001: 2010...2011...2012...2013...2014...